<a href="https://colab.research.google.com/github/VOX304/SchoolChatbot/blob/main/Preprocessing/Colab_preprocessing/Image_Prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install paddleocr ultralytics opencv-python numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   

In [ ]:
!pip install pdf2image python-docx pdfplumber paddlepaddle-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 759.0/759.0 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 73.5 MB/s eta 0:00:00
  Attempting uninstall: opt-einsum
    Found existing installation: opt_einsum 3.4.0
    Uninstalling opt_einsum-3.4.0:
      Successfully uninstalled opt_einsum-3.4.0


In [ ]:
!pip install paddlepaddle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 MB 5.4 MB/s eta 0:00:00


In [ ]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 80.8 MB/s eta 0:00:00


In [ ]:
import os
import json
import fitz  # PyMuPDF
from paddleocr import PaddleOCR
from ultralytics import YOLO
from docx import Document

# Initialize models
ocr = PaddleOCR(use_angle_cls=True, lang='vi')  # Vietnamese OCR with angle correction
yolo = YOLO('yolov8s.pt')  # Load YOLOv8 small model


[2025/03/24 02:13:54] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/latin/latin_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_

In [ ]:

# Confidence thresholds
OCR_CONF_THRESH = 0.7  # Filter out OCR results below this
OBJ_CONF_THRESH = 0.5  # Filter out detected objects below this

def extract_images_from_pdf(pdf_path, output_folder="images"):
    """Extract images from a PDF file using PyMuPDF and store metadata with page numbers."""
    os.makedirs(output_folder, exist_ok=True)
    image_metadata = []

    doc = fitz.open(pdf_path)  # Open the PDF
    for page_num in range(len(doc)):
        for img_index, img in enumerate(doc[page_num].get_images(full=True)):
            xref = img[0]  # Image XREF
            img_data = doc.extract_image(xref)  # Extract image data
            img_bytes = img_data["image"]  # Get raw image bytes

            # Save the image
            img_path = os.path.join(output_folder, f"pdf_img_{page_num + 1}_{img_index}.png")
            with open(img_path, "wb") as f:
                f.write(img_bytes)

            # Store metadata for VectorDB
            image_metadata.append({
                "document_name": os.path.basename(pdf_path),
                "document_type": "pdf",
                "page_number": page_num + 1,
                "Data_type" : "image"
                "image_index": img_index,
                "image_path": img_path
            })

    return image_metadata

def extract_images_from_docx(docx_path, output_folder="images"):
    """Extract images from a DOCX file and store metadata."""
    os.makedirs(output_folder, exist_ok=True)
    image_metadata = []
    doc = Document(docx_path)

    for i, rel in enumerate(doc.part.rels.values()):
        if "image" in rel.reltype:
            img_data = rel.target_part.blob
            img_path = os.path.join(output_folder, f"docx_img_{i}.jpg")
            with open(img_path, "wb") as f:
                f.write(img_data)

            # Store metadata for VectorDB
            image_metadata.append({
                "document_name": os.path.basename(docx_path),
                "document_type": "docx",
                "Data_type" : "image"
                "image_index": i,
                "image_path": img_path
            })

    return image_metadata

def detect_text(image_path):
    """Apply PaddleOCR to extract text from an image with confidence filtering."""
    results = ocr.ocr(image_path, cls=True)
    extracted_text = []
    for line in results:
        for word_info in line:
            text, confidence = word_info[1][0], word_info[1][1]
            if confidence >= OCR_CONF_THRESH:
                extracted_text.append(text)
    return extracted_text

def detect_objects(image_path):
    """Apply YOLOv8 to detect objects in an image with confidence filtering."""
    results = yolo(image_path)
    detected_objects = []
    for result in results:
        for box in result.boxes.data:
            conf = float(box[4])  # Confidence score
            if conf >= OBJ_CONF_THRESH:
                class_id = int(box[5])
                label = result.names[class_id]
                detected_objects.append(label)
    return detected_objects

def process_image(image_path):
    """Determine if an image contains text or objects and process accordingly."""
    text_results = detect_text(image_path)
    object_results = detect_objects(image_path)

    metadata = {
        "image_path": image_path,
        "extracted_text": text_results if text_results else None,
        "detected_objects": object_results if object_results else None
    }
    return metadata

def process_documents_in_folder(document_folder, output_json="image_metadata.json"):
    """Process all PDF and DOCX files in a folder and save metadata."""
    all_metadata = []
    os.makedirs(document_folder, exist_ok=True)

    for file_name in os.listdir(document_folder):
        file_path = os.path.join(document_folder, file_name)
        if file_name.endswith(".pdf"):
            image_metadata = extract_images_from_pdf(file_path)
        elif file_name.endswith(".docx"):
            image_metadata = extract_images_from_docx(file_path)
        else:
            continue  # Skip unsupported file types

        for img_data in image_metadata:
            img_path = img_data["image_path"]
            processed_metadata = process_image(img_path)
            img_data.update(processed_metadata)  # Merge extracted image data with metadata
            all_metadata.append(img_data)

    # Save metadata to JSON for easy VectorDB insertion
    with open(output_json, "w", encoding="utf-8") as f:
        json.dump(all_metadata, f, ensure_ascii=False, indent=4)

    return all_metadata

# Example usage
document_folder = "/content/sample_data/tcu"  # Replace with your folder path
metadata_results = process_documents_in_folder(document_folder)
print("✅ Metadata saved to image_metadata.json")


[2025/03/24 02:14:01] ppocr DEBUG: dt_boxes num : 41, elapsed : 0.44896531105041504
[2025/03/24 02:14:02] ppocr DEBUG: cls num  : 41, elapsed : 0.14949822425842285
[2025/03/24 02:14:06] ppocr DEBUG: rec_res num  : 41, elapsed : 4.698275804519653

image 1/1 /content/images/pdf_img_1_0.png: 640x480 1 book, 72.5ms
Speed: 7.5ms preprocess, 72.5ms inference, 373.8ms postprocess per image at shape (1, 3, 640, 480)
[2025/03/24 02:14:12] ppocr DEBUG: dt_boxes num : 43, elapsed : 0.3224635124206543
[2025/03/24 02:14:13] ppocr DEBUG: cls num  : 43, elapsed : 0.22592997550964355
[2025/03/24 02:14:17] ppocr DEBUG: rec_res num  : 43, elapsed : 4.603851795196533

image 1/1 /content/images/pdf_img_2_0.png: 640x480 (no detections), 12.3ms
Speed: 3.0ms preprocess, 12.3ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 480)
[2025/03/24 02:14:18] ppocr DEBUG: dt_boxes num : 43, elapsed : 0.2019197940826416
[2025/03/24 02:14:18] ppocr DEBUG: cls num  : 43, elapsed : 0.12732481956481934
[2025/0

In [ ]:

import os

def check_files_in_folder(folder_path):
    """Kiểm tra thư mục có chứa file không"""
    if not os.path.exists(folder_path):
        print(f"⚠️ Thư mục '{folder_path}' không tồn tại!")
        return False

    files = os.listdir(folder_path)

    if not files:
        print(f"📂 Thư mục '{folder_path}' rỗng!")
        return False
    else:
        print(f"✅ Thư mục '{folder_path}' có {len(files)} tệp tin:")
        for file in files:
            print(f"   📄 {file}")
        return True

# 📌 Đường dẫn thư mục cần kiểm tra
folder_path = "/content/sample_data/tcu"
check_files_in_folder(folder_path)


✅ Thư mục '/content/sample_data/tcu' có 8 tệp tin:
   📄 Nội dung tuyên truyền_2025313935.docx
   📄 CV 623 Hướng dẫn sơ tuyển_20252211126.pdf
   📄 Tuyển sinh trình độ đại học hệ dân sự_2025313935.docx
   📄 VB hợp nhất TT 31_20252211126.pdf
   📄 Địa chỉ trang mạng_2025313935.docx
   📄 2. Hỏi - Đáp tuyển sinh 2024_20252211126.docx
   📄 Đề án tuyển sinh Trường SQTT 2025_2025313935.docx
   📄 HD1860 công tác xét tuyển đào tạo ĐH, CĐ năm 2024_20252211126.pdf


True